In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import urllib.parse
import base64

# Import CRUD module
from crud_operations import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# MongoDB authentication
username = urllib.parse.quote_plus("aacuser")
password = urllib.parse.quote_plus("jade1234")

# Instantiate CRUD object
shelter = AnimalShelter(username, password)

# Retrieve all data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove '_id' column to avoid datatype issues
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo with hyperlink
image_filename = 'logo.png'  # Replace with actual image file
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    logo_image = html.A(
        href="https://www.snhu.edu",  # URL to link to SNHU
        target="_blank",  # Open in new tab
        children=[
            html.Img(
                src=f'data:image/png;base64,{encoded_image}',
                style={'height': '100px'}
            )
        ]
    )
except Exception:
    logo_image = html.H3("Logo not found. Please ensure 'logo.png' is in the correct path.")

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(logo_image),
    html.Hr(),

    # Interactive Filtering Options
    html.Label("Filter Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset/Initial State', 'value': 'reset'}
        ],
        value='reset',  # Ensures the dashboard starts with all data
        labelStyle={'display': 'block'}
    ),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),  # Loads full dataset at startup
        row_selectable="single",  # Enable single-row selection
        selected_rows=[0],  # Default to first row selected
        page_size=10,  # Limit rows displayed per page
        style_table={'overflowX': 'auto'},  # Allow scrolling if needed
        style_cell={'textAlign': 'left'}
    ),

    html.Br(),
    html.Hr(),

    # Layout for Charts & Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),

    # Unique Identifier
    html.H2("Your Unique Identifier: JADE PINEDA")
])

#############################################
# Interaction Between Components / Controller
#############################################

# Function to filter database queries
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}

    if filter_type == 'water':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}

    elif filter_type == 'mountain':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}

    elif filter_type == 'disaster':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}

    # Load ALL data explicitly on reset
    if filter_type == 'reset' or filter_type is None:
        query = {}

    # Fetch filtered data
    filtered_data = pd.DataFrame.from_records(shelter.read(query))
    if "_id" in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)

    return filtered_data.to_dict('records')

# Callback to generate a pie chart based on breed count
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return html.P("No data available.")

    dff = pd.DataFrame.from_dict(viewData)
    
    if "breed" in dff.columns and not dff.empty:
        fig = px.pie(dff, names='breed', title='Breed Distribution')
        return dcc.Graph(figure=fig)

    return html.P("No breed data available.")

# Callback to update map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or index is None or len(index) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10,
                      children=[dl.TileLayer(id="base-layer-id")])

    # Convert dictionary data to dataframe
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]  # Get selected row index

    if "location_lat" in dff.columns and "location_long" in dff.columns and not dff.empty:
        return dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                          children=[
                              dl.Tooltip(dff.iloc[row]['breed']),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row]['name'])
                              ])
                          ])
            ]
        )

    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10,
                  children=[dl.TileLayer(id="base-layer-id")])

# Run app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:12136/
